# Install Java, Spark, and Findspark
This installs Apache Spark 3.0.1, Java 11, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget http://www-eu.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install findspark
!pip install py4j
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init()
print("Initialization Finished.")
#!cat /proc/meminfo
#!cat /proc/cpuinfo

--2020-11-10 22:35:02--  http://www-eu.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
Resolving www-eu.apache.org (www-eu.apache.org)... 95.216.24.32, 2a01:4f9:2a:185f::2
Connecting to www-eu.apache.org (www-eu.apache.org)|95.216.24.32|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz [following]
--2020-11-10 22:35:02--  https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219929956 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.1-bin-hadoop2.7.tgz’

spark-3.0.1-bin-had 100%[===================>] 209.74M  9.94MB/s    in 22s     

2020-11-10 22:35:25 (9.56 MB/s) - ‘spark-3.0.1-bin-hadoop2.7.tgz’ saved [21992

#Hold connection
While not using sheet, you should start infinite loop function to hold session. If you want to execute another code, please interupt this function.

In [ ]:
import time
try:
  while 1:
    time.sleep(10)
except:
  pass

#Upload files

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cars.txt to cars.txt


#Generate random stream
Generator generates car brand and number of cars sold

In [ ]:
import threading
import random
import socket
import sys
import time

class WordGeneratorThread (threading.Thread):
    def __init__(self):
      threading.Thread.__init__(self)
      self.is_running = True

    def run(self):
      file = open('cars.txt', 'r')
      cars = file.readlines()

      HOST = 'localhost'  # Standard loopback interface address (localhost)
      PORT = 9999        # Port to listen on (non-privileged ports are > 1023)

      with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
          s.bind((HOST, PORT))
          s.listen(5)
          print('Words generator is listening')
          while True:
              conn, addr = s.accept()
              conn.send((cars[random.randint(5, 8)].strip('\n')+"_"+str(random.randint(1, 10))+"\n").encode("utf-8"))
              conn.send((cars[random.randint(7, 11)].strip('\n')+"_"+str(random.randint(1, 10))+"\n").encode("utf-8"))
              conn.send((cars[random.randint(0, len(cars)-1)].strip('\n')+"_"+str(random.randint(1, 10))+"\n").encode("utf-8"))
              conn.close()
              time.sleep(0.2)
              if not self.is_running:
                s.close()
                break

wordGeneratorThreadread = WordGeneratorThread()
wordGeneratorThreadread.setDaemon(True)
wordGeneratorThreadread.start()

Words generator is listening


#Exercise

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
sc = SparkContext(appName="Exercise")


# Start streaming context, batches each 5 seconds
ssc = .....
# connect to stream with format carBrand_numSoldCars - "Volvo_5"
lines = .....
# Map car brand and number of sold car to Pair by splitting input string
pairs = lines. .....
# Map pairs again and change number value of sold cars from string to int
pairs = .....
# Count numbers of sold cars for every brand
wordCounts = pairs. .....
# Update running counts
runningCounts = wordCounts.updateStateByKey(.....)
# If car count will increse 20, filter it and print stream
filtered = runningCounts .....
filtered.pprint()
# If car count will be lower than 20, filter it and print stream
finished = runningCounts. .....
finished.pprint()

# Set checkpoint and start
ssc.checkpoint('checkpoint.data')
ssc.start()
ssc.awaitTermination()

#Solution

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
sc = SparkContext(appName="Exercise")

def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount) 

# Start streaming context, batches each 5 seconds
ssc = StreamingContext(sc, 5)
# connect to stream with format carBrand_numSoldCars - "Volvo_5"
lines = ssc.socketTextStream('localhost', 9999)
# Map car brand and number of sold car to Pair by splitting input string
pairs = lines.map(lambda line: (line.split("_")))
# Map pairs again and change number value of sold cars from string to int
pairs = pairs.map(lambda pair: (pair[0], int(pair[1])))
# Count numbers of sold cars for every brand
wordCounts = pairs.reduceByKey(lambda x, y: x + y)
# Update running counts
runningCounts = wordCounts.updateStateByKey(updateFunction)
# If car count will increse 20, filter it and print stream
filtered = runningCounts.filter(lambda x: x[1]<20)
filtered.pprint()
# If car count will be lower than 20, filter it and print stream
finished = runningCounts.filter(lambda x: x[1]>20)
finished.pprint()

# Set checkpoint and start
ssc.checkpoint('checkpoint.data')
ssc.start()
ssc.awaitTermination()

#Stop StreamingContext

In [ ]:
# Stop thread running in background
# wordGeneratorThreadread.is_running = False
# wordGeneratorThreadread.join()
# if ('s' in globals()):
#   s.close()
#   del globals()['s']

if ('ssc' in globals()):
  ssc.stop()
  del globals()['ssc']
if ('wordBlacklist' in globals()):
  del globals()['wordBlacklist']
if ('droppedWordsCounter' in globals()):
  del globals()['droppedWordsCounter']
if ('query' in globals()):
  query.stop()
  del globals()['query']
if ('spark' in globals()):
  spark.stop()
  del globals()['spark']

-------------------------------------------
Time: 2020-11-10 23:18:35
-------------------------------------------
('Ford', 10)
('Seat', 1)
('Peugeot', 17)
('Volvo', 10)
('Citroën', 9)
('Nissan', 14)
('Toyota', 12)
('Škoda', 19)
('Subaru', 5)

-------------------------------------------
Time: 2020-11-10 23:18:35
-------------------------------------------
('Land Rover', 65)
('Porsche', 28)
('Mercedes-Benz', 41)
('Volkswagen', 23)
('Jeep', 142)
('Kia', 122)
('Mazda', 95)
('Honda', 136)
('Audi', 24)
('Hyundai', 78)
...

-------------------------------------------
Time: 2020-11-10 23:18:40
-------------------------------------------
('Seat', 1)
('Peugeot', 17)
('Volvo', 10)
('Citroën', 9)
('Nissan', 14)
('Toyota', 18)
('Škoda', 19)
('Subaru', 5)

-------------------------------------------
Time: 2020-11-10 23:18:40
-------------------------------------------
('Land Rover', 71)
('Porsche', 28)
('Mercedes-Benz', 46)
('Volkswagen', 29)
('Jeep', 142)
('Kia', 154)
('Mazda', 95)
('Honda', 149)
(